In [1]:
import importlib.util

#import lancedb
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.vectorstores import lancedb
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
import importlib
import nltk
import ssl
import os
from langchain import HuggingFaceHub

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter



import textwrap

In [2]:
print(f"Current dir: {os.path.abspath(os.path.curdir)}")
# os.environ['OPENAI_API_KEY'] = ''
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''


Current dir: /Users/amlanchatterjee/PycharmProjects/rag-llm/com/5r/rag


In [3]:
#Disable ssl certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

#Download nltk modules if not there
if os.path.isfile("/Users/amlanchatterjee/nltk_data/corpora/wordnet.zip") is None:
    nltk.download('wordnet')

if os.path.isfile("/Users/amlanchatterjee/nltk_data/tokenizers/punkt.zip") is None:
    nltk.download('punkt')

pdf_folder_path="data"
loader = PyPDFDirectoryLoader(pdf_folder_path)
raw_pdf_doc = loader.load()

In [4]:
print("Splitting PDF Document text..")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=16)
texts = text_splitter.split_documents(raw_pdf_doc)
print(f"Document splitting completed.. [{texts.__len__()}] splits")
##print(f"{texts}")

Splitting PDF Document text..
Document splitting completed.. [303] splits


In [5]:
print("Loading Embeddings...")

Loading Embeddings...


In [ ]:
# # select which embeddings we want to use
# # embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
embeddings = HuggingFaceEmbeddings()
#embeddings = OpenAIEmbeddings()
print("Embeddings loaded")
#
# # create the vector store to use for indexing
# dbhost=""
# user=""
# url="redis://" + user + "@" + dbhost
# vectordb = Redis.from_documents(texts, embeddings, redis_url=url, index_name="link")
# vectordb = FAISS.load_local(folder_path="data", embeddings=embeddings, index_name='index', allow_dangerous_deserialization=True)
#vectordb = FAISS.from_texts(texts, embeddings)
#vectordb = Chroma.from_documents(texts, embeddings)
#
vectordb = lancedb.LanceDB.from_documents(texts, embeddings)
print(f"db created and loaded with split documents {vectordb}")
#query = "How to see extended capture data in X2D?"
query = "How to use Gradient tool in phocus?"
#
#docs_and_scores = db.similarity_search(query)

#embedding_vector = OpenAIEmbeddings().embed_query(query)
query_vec_embedding = embeddings.embed_query(query)
print(f"Query embedding vector created: {query_vec_embedding[0]} ...")
print(f"Query = {query}\n")
# result = db.similarity_search_with_relevance_scores(embedding_vector, embeddings)
# result = db.similarity_search_by_vector(embedding_vector, embeddings)
# result = await vectordb.asimilarity_search_by_vector(query_vec_embedding)

## WORKS
result = await vectordb.asimilarity_search(query)
print(f"Result: {result[0].page_content.splitlines(keepends=False)}\n")
# print(f"Result: {result[1].page_content.splitlines(keepends=False)}\n")
# for i in range(len(result)):
#     print(f"Result: {result[i].page_content.splitlines(keepends=False)}")

#####
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":1})
# print(f"Retriever: {retriever.__pretty__}")
#
print("downloading LLM model")

try:
    llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":1, "max_length":1024})

except Exception as e:
    print({e})

print("Successfully Loaded llm model")

print("Loading QA...")
try:
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
    ans = qa({"query": "query"})
    print(f"Answer recewived is \n f{ans}")
except Exception as x:
    print({x})

print("QA answered")


Embeddings loaded
db created and loaded with split documents <langchain_community.vectorstores.lancedb.LanceDB object at 0x3203621d0>
Query embedding vector created: 0.021041328087449074 ...
Query = How to use Gradient tool in phocus?

Result: ['USER GUIDEwww.hasselblad.com', 'Phocus82', 'Table of ContentsGradient Tool', '1. Add a new adjustment layer  and select the ', 'Gradient tool. ', '2. Select the Gradient tool  and drag from where the ', 'water meets the rocks and up a bit. The gradient ', 'can be adjusted later using the handles on the ', 'gradient. See #4.  ', '3. Add a new layer by clicking the "+". Set Saturation  ', 'to a negative value to decrease saturation. Drag a ', 'new gradient. This time in the opposite direction. ', 'Fine-tune the setting for Saturation for the desired ', 'result.', '4.  Gradient manupulation handles:  ', ' ', 'A: The white outline indicates active side. Modify ', 'the gradient by moving this handle.  ', ' ', 'B: Centre handle - Moves the gradient. 

/Users/amlanchatterjee/PycharmProjects/rag-llm/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/Users/amlanchatterjee/PycharmProjects/rag-llm/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Successfully Loaded llm model
Loading QA...
